# CERT + OpenLLMetry Integration

This notebook demonstrates how to use **OpenLLMetry** to automatically send LLM traces to the **CERT Dashboard** for evaluation and monitoring.

## Architecture
```
┌─────────────────────────────────────────────────────────────────┐
│  Your Application (this notebook, or any app)                  │
│                                                                 │
│  Traceloop.init()  ← ONE LINE instruments everything           │
│                                                                 │
│  Automatically traces:                                          │
│  ✓ OpenAI, Anthropic, Cohere, Gemini, Mistral, Bedrock         │
│  ✓ LangChain, LlamaIndex, CrewAI, Haystack                     │
│  ✓ Pinecone, Chroma, Qdrant, Weaviate                          │
└────────────────────────┬────────────────────────────────────────┘
                         │ OTLP Protocol (automatic)
                         ▼
              ┌──────────────────────┐
              │   CERT Dashboard     │
              │   /api/v1/traces     │
              │                      │
              │  • View all traces   │
              │  • LLM Judge eval    │
              │  • Cost tracking     │
              │  • Performance       │
              └──────────────────────┘
```

## No API Keys Shared!
- Your API keys stay in YOUR environment
- Only trace data (prompts, responses, metrics) is sent to CERT
- CERT can be self-hosted for full privacy

## 1. Installation

In [ ]:
# Install OpenLLMetry and LLM SDKs
!pip install -q traceloop-sdk
!pip install -q openai anthropic
!pip install -q langchain langchain-openai langchain-anthropic

## 2. Configure CERT Endpoint

Set the OTLP endpoint to point to your CERT dashboard.

In [ ]:
import os
from getpass import getpass

# === CERT Dashboard Configuration ===
# For local development:
CERT_ENDPOINT = "http://localhost:3000/api/v1/traces"

# For deployed CERT:
# CERT_ENDPOINT = "https://your-cert-dashboard.com/api/v1/traces"

# Configure OpenTelemetry to send to CERT
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = CERT_ENDPOINT
os.environ["OTEL_EXPORTER_OTLP_PROTOCOL"] = "http/json"  # Use JSON for compatibility

# Optional: Add authentication header
# os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = "Authorization=Bearer your-token"

print(f"CERT endpoint configured: {CERT_ENDPOINT}")

In [ ]:
# Set your LLM API keys (these stay in YOUR environment)
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass('Enter your Anthropic API key: ')

print("API keys configured (stored locally only)")

## 3. Initialize OpenLLMetry

**ONE LINE** to instrument all your LLM calls!

In [ ]:
from traceloop.sdk import Traceloop

# Initialize OpenLLMetry - this instruments EVERYTHING
Traceloop.init(
    app_name="cert-demo",
    disable_batch=True,  # Send traces immediately (for demo)
    telemetry_enabled=False,  # Disable anonymous telemetry
)

print("✓ OpenLLMetry initialized")
print("  All LLM calls will now be traced and sent to CERT")

## 4. Make LLM Calls (Automatically Traced)

Now any LLM call you make will be automatically traced and sent to CERT.

### 4.1 Direct OpenAI Call

In [ ]:
from openai import OpenAI

client = OpenAI()

# This call is AUTOMATICALLY traced by OpenLLMetry
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful financial analyst."},
        {"role": "user", "content": "What are the key metrics to evaluate a company's financial health?"}
    ],
    temperature=0.7,
    max_tokens=500
)

print("OpenAI Response:")
print(response.choices[0].message.content)
print(f"\n→ Trace sent to CERT (tokens: {response.usage.total_tokens})")

### 4.2 Direct Anthropic Call

In [ ]:
import anthropic

claude = anthropic.Anthropic()

# This call is AUTOMATICALLY traced by OpenLLMetry
response = claude.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=500,
    messages=[
        {"role": "user", "content": "Explain the difference between P/E ratio and P/B ratio in 2 sentences."}
    ]
)

print("Claude Response:")
print(response.content[0].text)
print(f"\n→ Trace sent to CERT (tokens: {response.usage.input_tokens + response.usage.output_tokens})")

### 4.3 LangChain Calls

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

# Create chains with different models
gpt_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
claude_model = ChatAnthropic(model="claude-sonnet-4-5-20250929", temperature=0.3)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise financial advisor."),
    ("human", "{question}")
])

# Chain 1: GPT-4o
chain1 = prompt | gpt_model
result1 = chain1.invoke({"question": "What is dollar-cost averaging?"})
print("GPT-4o:", result1.content[:200], "...")

# Chain 2: Claude
chain2 = prompt | claude_model
result2 = chain2.invoke({"question": "What is dollar-cost averaging?"})
print("\nClaude:", result2.content[:200], "...")

print("\n→ Both traces sent to CERT automatically!")

### 4.4 Custom Workflow with Decorators

In [ ]:
from traceloop.sdk.decorators import workflow, task

@workflow(name="financial-analysis")
def analyze_company(company_name: str):
    """Multi-step analysis workflow - each step is traced"""
    
    # Step 1: Get company overview (GPT)
    overview = get_company_overview(company_name)
    
    # Step 2: Analyze risks (Claude)
    risks = analyze_risks(company_name, overview)
    
    return {
        "company": company_name,
        "overview": overview,
        "risks": risks
    }

@task(name="get-overview")
def get_company_overview(company: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"Give a 2-sentence overview of {company} as a business."}],
        max_tokens=100
    )
    return response.choices[0].message.content

@task(name="analyze-risks")
def analyze_risks(company: str, overview: str) -> str:
    response = claude.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=150,
        messages=[{"role": "user", "content": f"Based on this overview of {company}: '{overview}', list 2 key business risks."}]
    )
    return response.content[0].text

# Run the workflow
result = analyze_company("Apple Inc.")
print("Analysis Result:")
print(f"Company: {result['company']}")
print(f"Overview: {result['overview']}")
print(f"Risks: {result['risks']}")
print("\n→ Complete workflow traced with parent-child spans!")

## 5. View Traces in CERT Dashboard

Open your CERT dashboard to see all the traces we just sent!

In [ ]:
import requests

# Fetch traces from CERT
try:
    response = requests.get(f"{CERT_ENDPOINT}?llm_only=true&limit=10")
    if response.status_code == 200:
        data = response.json()
        print(f"CERT Dashboard Stats:")
        print(f"  Total traces: {data['stats']['total']}")
        print(f"  LLM traces: {data['stats']['llmTraces']}")
        print(f"  Total tokens: {data['stats']['totalTokens']:,}")
        print(f"  By vendor: {data['stats']['byVendor']}")
        print(f"\nRecent LLM calls:")
        for trace in data['traces'][:5]:
            if trace.get('llm'):
                print(f"  • {trace['llm']['vendor']}/{trace['llm']['model']} - {trace['llm']['totalTokens']} tokens")
    else:
        print(f"Could not fetch from CERT: {response.status_code}")
        print("Make sure your CERT dashboard is running!")
except requests.exceptions.ConnectionError:
    print("Could not connect to CERT dashboard.")
    print(f"Make sure it's running at: {CERT_ENDPOINT}")

## 6. Alternative: Direct CERT SDK (No OpenLLMetry)

If you prefer not to use OpenLLMetry, you can send traces directly to CERT.

In [ ]:
import requests
import time
from datetime import datetime

def send_to_cert(provider: str, model: str, input_text: str, output_text: str, 
                 prompt_tokens: int, completion_tokens: int, duration_ms: int):
    """Send a trace directly to CERT using the simple SDK format"""
    
    payload = {
        "traces": [{
            "id": f"manual-{int(time.time() * 1000)}",
            "provider": provider,
            "model": model,
            "input": input_text,
            "output": output_text,
            "promptTokens": prompt_tokens,
            "completionTokens": completion_tokens,
            "durationMs": duration_ms,
            "timestamp": datetime.now().isoformat()
        }]
    }
    
    response = requests.post(
        CERT_ENDPOINT,
        json=payload,
        headers={"Content-Type": "application/json"}
    )
    
    return response.json()

# Example: Manual trace
result = send_to_cert(
    provider="openai",
    model="gpt-4o",
    input_text="What is the capital of France?",
    output_text="The capital of France is Paris.",
    prompt_tokens=12,
    completion_tokens=8,
    duration_ms=450
)
print(f"Manual trace sent: {result}")

## Summary

### What We Demonstrated

| Feature | How It Works |
|---------|-------------|
| **Auto-instrumentation** | `Traceloop.init()` - one line, instruments all LLM calls |
| **Privacy** | API keys stay in your environment, only traces sent |
| **Multi-provider** | Works with OpenAI, Anthropic, and any supported provider |
| **Frameworks** | LangChain, LlamaIndex, and others auto-traced |
| **Custom workflows** | Use `@workflow` and `@task` decorators for complex flows |
| **Self-hosted** | CERT dashboard runs on your infrastructure |

### Next Steps

1. Open CERT dashboard → View traces
2. Run LLM Judge evaluations on collected traces
3. Monitor costs and performance metrics
4. Set up alerts for quality issues